In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import panel as pn

In [2]:

# Read transactions_2022_2023_categorized.csv
df = pd.read_csv('all_trans_end.csv')
# Add year and month columns
df['Year'] = pd.to_datetime(df['Date']).dt.year
df['Month'] = pd.to_datetime(df['Date']).dt.month
df['Month Name'] = pd.to_datetime(df['Date']).dt.strftime("%b")


# Remove "Transaction" and "Transaction vs category" columns
df = df.drop(columns=['Transaction', 'Transaction vs category','fname','Address','Reference Number'])
#if df['Amount'] < 0 : df['credit\debit']= 'debit'
#else : df['credit\debit']= 'credit'

#https://www.geeksforgeeks.org/ways-to-apply-an-if-condition-in-pandas-dataframe/
df['Amount'] = df['Amount'].fillna(0)
df.loc[df['Amount'] < 0, "credit/debit"] = 'Debit'
df.loc[df['Amount'] >= 0, "credit/debit"] = 'Credit'

df

,Date,Description,Amount,account,Category,Year,Month,Month Name,credit/debit
0,03/01/2024,BKOFAMERICA ATM 0301 XXXXX9075 WITHDRWL PENNSA...,-500.00,9443,Bank Withdrawal,2024,3,Mar,Debit
1,03/01/2024,COMCAST CC OF DESPAYROLL IDXXXXXXXXXX8070 INDN...,4189.35,9443,Cable/Internet,2024,3,Mar,Credit
2,03/01/2024,Beginning balance as of 03012024,0.00,4538,Balance Update,2024,3,Mar,Credit
3,03/01/2024,MOHELA DESQDR IDXXXXX19877 INDNWIERZCHOWSKI JO...,-1259.86,4538,Student Loan,2024,3,Mar,Debit
4,03/04/2024,NSM DBAMRCOOPER DESNSM DBAMR ID4652654 INDNJOH...,-988.73,9443,Credit Card Payment,2024,3,Mar,Debit
...,...,...,...,...,...,...,...,...,...
541,09/23/2024,Zelle payment from GREGORY WIERZCHOWSKI for WE...,104.38,9443,Household/Tools,2024,9,Sep,Credit
542,09/24/2024,Zelle payment to KERSAUMIE ROZAN Conf nuscydpkh,-155.55,9443,Zelle Payment/Misc,2024,9,Sep,Debit
543,09/24/2024,AMAZON MKTPLX44WE5K83 AmzncombillWA,-40.19,9400,Online Shopping/Groceries,2024,9,Sep,Debit
544,09/25/2024,Zelle payment from ROCHELLE L GRANT for Tickets,30.00,9443,Entertainment/Tickets,2024,9,Sep,Credit


In [3]:
def make_pie_chart(df, year, label):
    # Filter the dataset for expense transactions
    
    sub_df = df[(df["credit/debit"] == label) & (df['Year'] == year)]
    if label == 'Debit': 
        sub_df['Amount'] *= -1 
    color_scale = px.colors.qualitative.Set2
    
    pie_fig = px.pie(sub_df, values='Amount', names='Category', color_discrete_sequence = color_scale)
    pie_fig.update_traces(textposition='inside', direction ='clockwise', hole=0.3, textinfo="label+percent")
    
    total_expense = df[(df["credit/debit"] == 'Debit') & (df['Year'] == year)]['Amount'].sum() 
    total_income = df[(df["credit/debit"] == 'Credit') & (df['Year'] == year)]['Amount'].sum()
   
    if label == 'Credit':
        total_text = "$ " + str(round(total_income))
    
        # Saving rate:
        saving_rate = round((total_income - total_expense)/total_income*100)
        saving_rate_text = ": Saving rate " + str(saving_rate) + "%"
    else:
        saving_rate_text = ""
        total_text = "$ " + str(round(total_expense))
    
    pie_fig.update_layout(uniformtext_minsize=10, 
                        uniformtext_mode='hide',
                        title=dict(text=label+" Breakdown " + str(year) + saving_rate_text),
                        # Add annotations in the center of the donut.
                        annotations=[
                            dict(
                                text=total_text, 
                                # Square unit grid starting at bottom left of page
                                x=0.5, y=0.5, font_size=12,
                                # Hide the arrow that points to the [x,y] coordinate
                                showarrow=False
                            )
                        ]
                    )
    return pie_fig

In [4]:
def make_monthly_bar_chart(df, year, label):
    df = df[(df["credit/debit"] == label) & (df['Year'] == year)]
    total_by_month = (df.groupby(['Month', 'Month Name'])['Amount'].sum()
                        .to_frame()
                        .reset_index()
                        .sort_values(by='Month')  
                        .reset_index(drop=True))
    if label == "Credit":
        color_scale = px.colors.sequential.YlGn
    if label == "Debit":
        color_scale = px.colors.sequential.OrRd
    
    bar_fig = px.bar(total_by_month, x='Month Name', y='Amount', text_auto='.2s', title=label+" per month", color='Amount', color_continuous_scale=color_scale)
    # bar_fig.update_traces(marker_color='lightslategrey')
    
    return bar_fig

In [5]:
# Pie charts
income_pie_fig_2024 = make_pie_chart(df, 2024, 'Credit')
expense_pie_fig_2024 = make_pie_chart(df, 2024, 'Debit')  
# Bar charts
income_monthly_2024 = make_monthly_bar_chart(df, 2024, 'Credit')
expense_monthly_2024 = make_monthly_bar_chart(df, 2024, 'Debit')




tabs = pn.Tabs(
                       ('2024', pn.Column(pn.Row(income_pie_fig_2024, expense_pie_fig_2024),
                                                pn.Row(income_monthly_2024, expense_monthly_2024))
                        )
                )
#tabs.show()

/tmp/ipykernel_17887/434039115.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [6]:
# Dashboard template
template = pn.template.FastListTemplate(
    title='Personal Finance Dashboard',
    sidebar=[pn.pane.Markdown("# Income Expense analysis"), 
             pn.pane.Markdown("Overview of income and expense based on my bank transactions. Categories are obtained using local LLMs."),
             pn.pane.PNG("picture.png", sizing_mode="scale_both")
             ],
    main=[pn.Row(pn.Column(pn.Row(tabs)
                           )
                ),
                ],
    # accent_base_color="#88d8b0",
    header_background="#c0b9dd",
)

template.show()

Launching server at http://localhost:42883
